# Twitter Sentiment Analysis

In [45]:
import pandas as pd
import numpy as np
import nltk
from bs4 import BeautifulSoup
import string
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
import re
import matplotlib.pyplot as plt 
import seaborn as sns
import string
import warnings 
warnings.filterwarnings("ignore", category=DeprecationWarning)

%matplotlib inline

In [46]:
df = pd.read_csv('cruiselines_all_hashtags_july.csv')

In [47]:
df.head()

,Unnamed: 0,date,username,tweet,replies_count,retweets_count,likes_count,hashtags,link,Cruiseline
0,0,7/7/20,tboholidays,Enjoy unbeatable savings and explore your dream destination alongside your favorite travel companion.\n\nBook Now: https://www.tboholidays.com/azamara.aspx \n#cruises #azamaracruise #cruiseholiday #cruisevacation #cruisesailing #vacation #holidays #tboholidays #cruisetrip pic.twitter.com/YwlLxyMb11,0,0,0,"['#cruises', '#azamaracruise', '#cruiseholiday', '#cruisevacation', '#cruisesailing', '#vacation', '#holidays', '#tboholidays', '#cruisetrip']",https://twitter.com/tboholidays/status/1280425329241329667,Azamara
1,1,7/1/20,cruisep29384190,"There's never been a keener, CLEANER time to cruise. As always, we can show you the best way to do that. #travel #cruising #azamaracruise #vacations #cleaner #adventure #travelexpert #luxurytravel #cruise #cleancruising #bubbles #omahatraveladvisor #weknowbecausewego pic.twitter.com/lPrCH1Txwl",0,0,1,"['#travel', '#cruising', '#azamaracruise', '#vacations', '#cleaner', '#adventure', '#travelexpert', '#luxurytravel', '#cruise', '#cleancruising', '#bubbles', '#omahatraveladvisor', '#weknowbecausewego']",https://twitter.com/Cruisep29384190/status/1278339310832488450,Azamara
2,2,7/30/20,hiclassmag,Azamara Experiences High Demand for 2021 European Sailings #travel https://cruiseaddicts.com/azamara-experiences-high-demand-for-2021/ …,0,0,0,['#travel'],https://twitter.com/HiClassMag/status/1288950662433210370,Azamara
3,3,7/30/20,duggo41,"As a several times Azamara paying guest you can forget about me if you have staff wearing masks, social distancing and temperature controls.",0,0,0,[],https://twitter.com/duggo41/status/1288926299407491075,Azamara
4,4,7/30/20,murrayalanlee,Could you ask them where my shore excursions refunds are? @Mycomplaintonl3,0,0,0,[],https://twitter.com/MurrayAlanLee/status/1288910923491672064,Azamara


In [48]:
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

In [7]:
#df['company'] = 'All'

In [49]:
cols = ['date','username','tweet','replies_count','retweets_count','likes_count','hashtags','link','Cruiseline']
alldf = df[cols]

In [50]:
alldf['date'] = pd.to_datetime(alldf['date'], errors='coerce')

In [142]:
#df = alldf[alldf['date'] >= '2019-11-01']

In [51]:
alldf.head()

,date,username,tweet,replies_count,retweets_count,likes_count,hashtags,link,Cruiseline
0,2020-07-07,tboholidays,Enjoy unbeatable savings and explore your dream destination alongside your favorite travel companion.\n\nBook Now: https://www.tboholidays.com/azamara.aspx \n#cruises #azamaracruise #cruiseholiday #cruisevacation #cruisesailing #vacation #holidays #tboholidays #cruisetrip pic.twitter.com/YwlLxyMb11,0,0,0,"['#cruises', '#azamaracruise', '#cruiseholiday', '#cruisevacation', '#cruisesailing', '#vacation', '#holidays', '#tboholidays', '#cruisetrip']",https://twitter.com/tboholidays/status/1280425329241329667,Azamara
1,2020-07-01,cruisep29384190,"There's never been a keener, CLEANER time to cruise. As always, we can show you the best way to do that. #travel #cruising #azamaracruise #vacations #cleaner #adventure #travelexpert #luxurytravel #cruise #cleancruising #bubbles #omahatraveladvisor #weknowbecausewego pic.twitter.com/lPrCH1Txwl",0,0,1,"['#travel', '#cruising', '#azamaracruise', '#vacations', '#cleaner', '#adventure', '#travelexpert', '#luxurytravel', '#cruise', '#cleancruising', '#bubbles', '#omahatraveladvisor', '#weknowbecausewego']",https://twitter.com/Cruisep29384190/status/1278339310832488450,Azamara
2,2020-07-30,hiclassmag,Azamara Experiences High Demand for 2021 European Sailings #travel https://cruiseaddicts.com/azamara-experiences-high-demand-for-2021/ …,0,0,0,['#travel'],https://twitter.com/HiClassMag/status/1288950662433210370,Azamara
3,2020-07-30,duggo41,"As a several times Azamara paying guest you can forget about me if you have staff wearing masks, social distancing and temperature controls.",0,0,0,[],https://twitter.com/duggo41/status/1288926299407491075,Azamara
4,2020-07-30,murrayalanlee,Could you ask them where my shore excursions refunds are? @Mycomplaintonl3,0,0,0,[],https://twitter.com/MurrayAlanLee/status/1288910923491672064,Azamara


In [52]:
alldf = alldf.dropna()

In [53]:
alldf.isna().sum()

date              0
username          0
tweet             0
replies_count     0
retweets_count    0
likes_count       0
hashtags          0
link              0
Cruiseline        0
dtype: int64

In [54]:
df[df.duplicated()]

,date,username,tweet,replies_count,retweets_count,likes_count,hashtags,link,Cruiseline


In [58]:
df_clean = alldf.drop_duplicates()

#### Tweets Preprocessing

In [59]:
df = df_clean.copy()

In [60]:
df["tweet"] = df["tweet"].astype(str)
df["tweet"] = df["tweet"].str.lower()

In [61]:
len(df)

10029

In [62]:
print(df['date'].max())
print(df['date'].min())

2020-07-30 00:00:00
2020-07-01 00:00:00


#### Remove punctuations and urls

In [63]:
def repword(text):
    text = re.sub(r"pic.twitter.com", '', text) #Removing @mentions
    return text

In [64]:
def cleanTxt(text):
    text = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', text) #Removing @mentions
    return text

In [65]:
df['tidy_tweet'] = df['tweet'].apply(repword)

In [66]:
df['tidy_tweet'] = df['tweet'].apply(cleanTxt)

In [67]:
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

df['tidy_tweet'] = df['tidy_tweet'].apply(lambda text: remove_punctuation(text))

#### Filter Tweets in english

In [ ]:
#df.head()

In [68]:
words = set(nltk.corpus.words.words())
def remove_newords(text):
    """custom function to remove non english words"""
    return " ".join(w for w in nltk.wordpunct_tokenize(str(text)) if w.lower() in words or not w.isalpha())

In [69]:
df["tidy_tweet"]= df["tidy_tweet"].apply(lambda text: remove_newords(text))

In [70]:
pd.set_option('display.max_colwidth', -1)
df

,date,username,tweet,replies_count,retweets_count,likes_count,hashtags,link,Cruiseline,tidy_tweet
0,2020-07-07,tboholidays,enjoy unbeatable savings and explore your dream destination alongside your favorite travel companion.\n\nbook now: https://www.tboholidays.com/azamara.aspx \n#cruises #azamaracruise #cruiseholiday #cruisevacation #cruisesailing #vacation #holidays #tboholidays #cruisetrip pic.twitter.com/ywllxymb11,0,0,0,"['#cruises', '#azamaracruise', '#cruiseholiday', '#cruisevacation', '#cruisesailing', '#vacation', '#holidays', '#tboholidays', '#cruisetrip']",https://twitter.com/tboholidays/status/1280425329241329667,Azamara,enjoy unbeatable and explore your dream destination alongside your favorite travel companion book now vacation pictwittercomywllxymb11
1,2020-07-01,cruisep29384190,"there's never been a keener, cleaner time to cruise. as always, we can show you the best way to do that. #travel #cruising #azamaracruise #vacations #cleaner #adventure #travelexpert #luxurytravel #cruise #cleancruising #bubbles #omahatraveladvisor #weknowbecausewego pic.twitter.com/lprch1txwl",0,0,1,"['#travel', '#cruising', '#azamaracruise', '#vacations', '#cleaner', '#adventure', '#travelexpert', '#luxurytravel', '#cruise', '#cleancruising', '#bubbles', '#omahatraveladvisor', '#weknowbecausewego']",https://twitter.com/Cruisep29384190/status/1278339310832488450,Azamara,theres never been a keener cleaner time to cruise as always we can show you the best way to do that travel cleaner adventure cruise pictwittercomlprch1txwl
2,2020-07-30,hiclassmag,azamara experiences high demand for 2021 european sailings #travel https://cruiseaddicts.com/azamara-experiences-high-demand-for-2021/ …,0,0,0,['#travel'],https://twitter.com/HiClassMag/status/1288950662433210370,Azamara,high demand for 2021 travel …
3,2020-07-30,duggo41,"as a several times azamara paying guest you can forget about me if you have staff wearing masks, social distancing and temperature controls.",0,0,0,[],https://twitter.com/duggo41/status/1288926299407491075,Azamara,as a several times paying guest you can forget about me if you have staff wearing social and temperature
4,2020-07-30,murrayalanlee,could you ask them where my shore excursions refunds are? @mycomplaintonl3,0,0,0,[],https://twitter.com/MurrayAlanLee/status/1288910923491672064,Azamara,could you ask them where my shore are mycomplaintonl3
5,2020-07-30,azamara,"those who love to cruise know that the friends you make along the way make your travels all the more special. for #internationalfriendshipday, here are cruise directors tony markey and amanda poulson performing ""amigos para siempre (friends for life)"": https://youtu.be/o0ro2xrf-ba",3,5,18,['#internationalfriendshipday'],https://twitter.com/Azamara/status/1288909733357932549,Azamara,those who love to cruise know that the you make along the way make your all the more special for here are cruise tony and para for life
6,2020-07-30,jetsettersblog,azamara cruises global promotion - \n\n https://jetsettersblog.com/azamara-cruises-global-promotion/ …,0,0,1,[],https://twitter.com/jetsettersblog/status/1288882038691500034,Azamara,global promotion …
7,2020-07-30,rcclnews,"introducing the new royal caribbean group! (replaces royal caribbean cruise line ltd. the parent company of royal caribbean intl, celebrity cruises, and azamara.) pic.twitter.com/x2drod5wur",0,2,10,[],https://twitter.com/RCCLNews/status/1288881612541906944,Azamara,the new royal group royal cruise line the parent company of royal celebrity and pictwittercomx2drod5wur
8,2020-07-30,abroadinnocents,good luck!,0,0,2,[],https://twitter.com/AbroadInnocents/status/1288866584346234885,Azamara,good luck
9,2020-07-30,seas_jr,actually i'm going to speak to trading standards to see what else i can do.,2,0,3,[],https://twitter.com/seas_jr/status/1288855121267433474,Azamara,actually going to speak to trading to see what else i can do


#### Remove stopwords

In [ ]:
#STOPWORDS = set(stopwords.words('english'))
#def remove_stopwords(text):
    #"""custom function to remove the stopwords"""
    #return " ".join([word for word in str(text).split() if word not in STOPWORDS])
#df["tidy_tweet"]= df["tidy_tweet"].apply(lambda text: remove_stopwords(text))

In [71]:
import spacy 
from spacy.lang.en.stop_words import STOP_WORDS

In [72]:
def remove_stopwords_spacy(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOP_WORDS])
df["tidy_tweet"]= df["tidy_tweet"].apply(lambda text: remove_stopwords_spacy(text))

#### Retain words with atleast 3 alphabets

In [73]:
df['tidy_tweet'] = df['tidy_tweet'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>2]))

#### Lemmatization

In [74]:
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
wordnet_map = {"N":wordnet.NOUN, "V":wordnet.VERB, "J":wordnet.ADJ, "R":wordnet.ADV}
def lemmatize_words(text):
    pos_tagged_text = nltk.pos_tag(text.split())
    return " ".join([lemmatizer.lemmatize(word, wordnet_map.get(pos[0], wordnet.NOUN)) for word, pos in pos_tagged_text])

df['tidy_tweet'] = df['tidy_tweet'].apply(lambda text: lemmatize_words(text))
df.head()

,date,username,tweet,replies_count,retweets_count,likes_count,hashtags,link,Cruiseline,tidy_tweet
0,2020-07-07,tboholidays,enjoy unbeatable savings and explore your dream destination alongside your favorite travel companion.\n\nbook now: https://www.tboholidays.com/azamara.aspx \n#cruises #azamaracruise #cruiseholiday #cruisevacation #cruisesailing #vacation #holidays #tboholidays #cruisetrip pic.twitter.com/ywllxymb11,0,0,0,"['#cruises', '#azamaracruise', '#cruiseholiday', '#cruisevacation', '#cruisesailing', '#vacation', '#holidays', '#tboholidays', '#cruisetrip']",https://twitter.com/tboholidays/status/1280425329241329667,Azamara,enjoy unbeatable explore dream destination alongside favorite travel companion book vacation pictwittercomywllxymb11
1,2020-07-01,cruisep29384190,"there's never been a keener, cleaner time to cruise. as always, we can show you the best way to do that. #travel #cruising #azamaracruise #vacations #cleaner #adventure #travelexpert #luxurytravel #cruise #cleancruising #bubbles #omahatraveladvisor #weknowbecausewego pic.twitter.com/lprch1txwl",0,0,1,"['#travel', '#cruising', '#azamaracruise', '#vacations', '#cleaner', '#adventure', '#travelexpert', '#luxurytravel', '#cruise', '#cleancruising', '#bubbles', '#omahatraveladvisor', '#weknowbecausewego']",https://twitter.com/Cruisep29384190/status/1278339310832488450,Azamara,there keener clean time cruise best way travel clean adventure cruise pictwittercomlprch1txwl
2,2020-07-30,hiclassmag,azamara experiences high demand for 2021 european sailings #travel https://cruiseaddicts.com/azamara-experiences-high-demand-for-2021/ …,0,0,0,['#travel'],https://twitter.com/HiClassMag/status/1288950662433210370,Azamara,high demand 2021 travel
3,2020-07-30,duggo41,"as a several times azamara paying guest you can forget about me if you have staff wearing masks, social distancing and temperature controls.",0,0,0,[],https://twitter.com/duggo41/status/1288926299407491075,Azamara,time pay guest forget staff wear social temperature
4,2020-07-30,murrayalanlee,could you ask them where my shore excursions refunds are? @mycomplaintonl3,0,0,0,[],https://twitter.com/MurrayAlanLee/status/1288910923491672064,Azamara,ask shore mycomplaintonl3


#### Remove frequent and rare words

In [75]:
from collections import Counter
cnt = Counter()
for text in df["tidy_tweet"].values:
    for word in text.split():
        cnt[word] += 1
        
cnt.most_common(100)

[('cruise', 2951),
 ('refund', 780),
 ('ship', 737),
 ('day', 736),
 ('travel', 697),
 ('book', 619),
 ('sail', 595),
 ('wait', 548),
 ('time', 518),
 ('new', 435),
 ('like', 435),
 ('carnival', 418),
 ('look', 411),
 ('go', 406),
 ('great', 372),
 ('2021', 358),
 ('princess', 352),
 ('money', 336),
 ('love', 331),
 ('line', 328),
 ('thanks', 322),
 ('flag', 322),
 ('year', 320),
 ('get', 314),
 ('good', 295),
 ('know', 294),
 ('royal', 286),
 ('people', 282),
 ('today', 273),
 ('sea', 270),
 ('need', 269),
 ('tell', 250),
 ('world', 241),
 ('covid19', 240),
 ('think', 237),
 ('take', 236),
 ('want', 228),
 ('thank', 220),
 ('credit', 217),
 ('come', 216),
 ('vacation', 215),
 ('2020', 213),
 ('soon', 208),
 ('beautiful', 206),
 ('cant', 206),
 ('check', 205),
 ('hope', 203),
 ('way', 200),
 ('work', 198),
 ('best', 190),
 ('right', 190),
 ('miss', 189),
 ('march', 189),
 ('future', 178),
 ('happy', 173),
 ('receive', 172),
 ('help', 166),
 ('try', 163),
 ('amaze', 161),
 ('don', 159),

In [76]:
FREQWORDS = set([w for (w, wc) in cnt.most_common(10)])
def remove_freqwords(text):
    """custom function to remove the frequent words"""
    return " ".join([word for word in str(text).split() if word not in {'i’m','tom','ships','ship','cruise','cruises',
                                                                        'ive','i’ve','youre','didn’t','etc','it’s','dont','don’t',
                                                                       'cruiselife','cruiseholiday','cruisenew','cruisetravel',
                                                                       'cruisevacation','luxurycruise','go','get','2021','movie',
                                                                       'dont','2020','carnivalcruise','carnival','princesscruises',
                                                                       'cruisenorwegian','royalcaribbean','halcruises',
                                                                       'crystalcruises','princesscruises','norwegian','cruisenorwegian',
                                                                       'disneycruise','seabourn','viking'}])

In [77]:
df['tidy_tweet'] = df['tidy_tweet'].apply(lambda text: remove_freqwords(text))
df.head()

,date,username,tweet,replies_count,retweets_count,likes_count,hashtags,link,Cruiseline,tidy_tweet
0,2020-07-07,tboholidays,enjoy unbeatable savings and explore your dream destination alongside your favorite travel companion.\n\nbook now: https://www.tboholidays.com/azamara.aspx \n#cruises #azamaracruise #cruiseholiday #cruisevacation #cruisesailing #vacation #holidays #tboholidays #cruisetrip pic.twitter.com/ywllxymb11,0,0,0,"['#cruises', '#azamaracruise', '#cruiseholiday', '#cruisevacation', '#cruisesailing', '#vacation', '#holidays', '#tboholidays', '#cruisetrip']",https://twitter.com/tboholidays/status/1280425329241329667,Azamara,enjoy unbeatable explore dream destination alongside favorite travel companion book vacation pictwittercomywllxymb11
1,2020-07-01,cruisep29384190,"there's never been a keener, cleaner time to cruise. as always, we can show you the best way to do that. #travel #cruising #azamaracruise #vacations #cleaner #adventure #travelexpert #luxurytravel #cruise #cleancruising #bubbles #omahatraveladvisor #weknowbecausewego pic.twitter.com/lprch1txwl",0,0,1,"['#travel', '#cruising', '#azamaracruise', '#vacations', '#cleaner', '#adventure', '#travelexpert', '#luxurytravel', '#cruise', '#cleancruising', '#bubbles', '#omahatraveladvisor', '#weknowbecausewego']",https://twitter.com/Cruisep29384190/status/1278339310832488450,Azamara,there keener clean time best way travel clean adventure pictwittercomlprch1txwl
2,2020-07-30,hiclassmag,azamara experiences high demand for 2021 european sailings #travel https://cruiseaddicts.com/azamara-experiences-high-demand-for-2021/ …,0,0,0,['#travel'],https://twitter.com/HiClassMag/status/1288950662433210370,Azamara,high demand travel
3,2020-07-30,duggo41,"as a several times azamara paying guest you can forget about me if you have staff wearing masks, social distancing and temperature controls.",0,0,0,[],https://twitter.com/duggo41/status/1288926299407491075,Azamara,time pay guest forget staff wear social temperature
4,2020-07-30,murrayalanlee,could you ask them where my shore excursions refunds are? @mycomplaintonl3,0,0,0,[],https://twitter.com/MurrayAlanLee/status/1288910923491672064,Azamara,ask shore mycomplaintonl3


In [78]:
n_rare_words = 1000
RAREWORDS = set([w for (w, wc) in cnt.most_common()[:-n_rare_words-1:-1]])

In [79]:
n_rare_words = 1000
RAREWORDS = set([w for (w, wc) in cnt.most_common()[:-n_rare_words-1:-1]])

{'pictwittercomx95roucjd9',
 'killer',
 'traitorous',
 'deduct',
 'pictwittercomedp8yflq7k',
 'angle',
 'pictwittercomltnbrcxfr1',
 'expectation',
 'dealt',
 '❤❤❤❤❤',
 'unnecessary',
 'exceed',
 '9wxf59',
 'pictwittercomw5brtbtnfr',
 'pictwittercomhysmynmk5h',
 'pictwittercomrpbb2kehd5',
 'pictwittercomt4cbn7l3oy',
 'pictwittercomjtajsqy4bi',
 'correspondence',
 '🤦🏻\u200d♀️🙄',
 'destroyer',
 'wjxt4',
 'rat',
 '20202021',
 'pictwittercomwnbve92t8k',
 'marlinsrule1115',
 'don53benz',
 'pictwittercom1ochqmwdnv',
 'pictwittercomjy0ynhqyfc',
 'pictwittercomnlzihy4rrg',
 '373',
 'pictwittercomxka09iuwu7',
 'pictwittercomj1808m6dnu',
 'pictwittercom3lxzmukd8s',
 'cure',
 'bap',
 'pictwittercomhidf3j3ujc',
 'pictwittercom4orxxecxfw',
 'pictwittercomc4gryqxo4i',
 'pictwittercomv1shs6ugpx',
 'southwest',
 'numberm207',
 'pictwittercomqadn4jdlby',
 'outdated',
 'pictwittercomoxdbe61leo',
 '10pm',
 'utilize',
 'entitlement',
 'built',
 'compress',
 '046',
 'pictwittercomyyfwev3svz',
 'firing',
 '0

In [80]:
def remove_rarewords(text):
    """custom function to remove the rare words"""
    return " ".join([word for word in str(text).split() if word not in RAREWORDS])

df['tidy_tweet'] = df['tidy_tweet'].apply(lambda text: remove_rarewords(text))

#### Emoticons

In [34]:
def convert_emojis(text):
    for emot in UNICODE_EMO:
        text = text.replace(emot, "_".join(UNICODE_EMO[emot].replace(",","").replace(":","").split()))
    return text

In [31]:
df["tidy_tweet"] = df["tidy_tweet"].apply(lambda text: convert_emojis(text))

NameError: name 'UNICODE_EMO' is not defined

#### Tokenize Tweets and create new column

In [ ]:
df["token_tweet"] = df['tidy_tweet'].apply(lambda x: x.split())

#### Sentiment Classification

In [81]:
from textblob import TextBlob
def getPolarity(text):
    return  TextBlob(text).sentiment.polarity

In [82]:
df['Polarity'] = df['tidy_tweet'].apply(getPolarity)

In [83]:
def getAnalysis(score):
    if score < 0:
        return 'Negative'
    elif score == 0:
        return 'Neutral'
    else:
        return 'Positive'

In [84]:
df['Analysis'] = df['Polarity'].apply(getAnalysis)

In [85]:
df.head()

,date,username,tweet,replies_count,retweets_count,likes_count,hashtags,link,Cruiseline,tidy_tweet,Polarity,Analysis
0,2020-07-07,tboholidays,enjoy unbeatable savings and explore your dream destination alongside your favorite travel companion.\n\nbook now: https://www.tboholidays.com/azamara.aspx \n#cruises #azamaracruise #cruiseholiday #cruisevacation #cruisesailing #vacation #holidays #tboholidays #cruisetrip pic.twitter.com/ywllxymb11,0,0,0,"['#cruises', '#azamaracruise', '#cruiseholiday', '#cruisevacation', '#cruisesailing', '#vacation', '#holidays', '#tboholidays', '#cruisetrip']",https://twitter.com/tboholidays/status/1280425329241329667,Azamara,enjoy unbeatable explore dream destination alongside favorite travel companion book vacation pictwittercomywllxymb11,0.450000,Positive
1,2020-07-01,cruisep29384190,"there's never been a keener, cleaner time to cruise. as always, we can show you the best way to do that. #travel #cruising #azamaracruise #vacations #cleaner #adventure #travelexpert #luxurytravel #cruise #cleancruising #bubbles #omahatraveladvisor #weknowbecausewego pic.twitter.com/lprch1txwl",0,0,1,"['#travel', '#cruising', '#azamaracruise', '#vacations', '#cleaner', '#adventure', '#travelexpert', '#luxurytravel', '#cruise', '#cleancruising', '#bubbles', '#omahatraveladvisor', '#weknowbecausewego']",https://twitter.com/Cruisep29384190/status/1278339310832488450,Azamara,there keener clean time best way travel clean adventure pictwittercomlprch1txwl,0.577778,Positive
2,2020-07-30,hiclassmag,azamara experiences high demand for 2021 european sailings #travel https://cruiseaddicts.com/azamara-experiences-high-demand-for-2021/ …,0,0,0,['#travel'],https://twitter.com/HiClassMag/status/1288950662433210370,Azamara,high demand travel,0.160000,Positive
3,2020-07-30,duggo41,"as a several times azamara paying guest you can forget about me if you have staff wearing masks, social distancing and temperature controls.",0,0,0,[],https://twitter.com/duggo41/status/1288926299407491075,Azamara,time pay guest forget staff wear social temperature,0.033333,Positive
4,2020-07-30,murrayalanlee,could you ask them where my shore excursions refunds are? @mycomplaintonl3,0,0,0,[],https://twitter.com/MurrayAlanLee/status/1288910923491672064,Azamara,ask shore mycomplaintonl3,0.000000,Neutral


In [40]:
#df['company'] = 'All'

In [86]:
df.to_csv('all_cruiseline_cleaned_tweets_july.csv')

In [639]:
#df1= pd.read_csv('HAL_Cleaned_tweets.csv')
#df2 = pd.read_csv('Carnival_Cleaned_tweets.csv')

In [640]:
#df3 = pd.concat([df1,df2])

In [641]:
#df3.to_csv('Cruiselines_Cleaned_tweets.csv')

#### Weekly data

In [390]:
weeks = [g.reset_index()
         for n, g in df.set_index('date').groupby(pd.TimeGrouper('W'))]

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: pd.TimeGrouper is deprecated and will be removed; Please use pd.Grouper(freq=...)
  


In [ ]:
file_name_parts = "week_files/data_part_"
for i in range(len(weeks)):
    weeks[i].to_csv(file_name_parts + str(i).zfill(3) + ".csv", sep=',')

#### Term frequency count

In [581]:
df = weeks[28]

In [582]:
from sklearn.feature_extraction.text import CountVectorizer
cvec = CountVectorizer()

In [583]:
cvec.fit(df.tidy_tweet)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [584]:
neg_doc_matrix = cvec.transform(df[df.Analysis == 'Negative'].tidy_tweet)
pos_doc_matrix = cvec.transform(df[df.Analysis == 'Positive'].tidy_tweet)
neu_doc_matrix = cvec.transform(df[df.Analysis == 'Neutral'].tidy_tweet)
neg_tf = np.sum(neg_doc_matrix,axis=0)
pos_tf = np.sum(pos_doc_matrix,axis=0)
neu_tf = np.sum(neu_doc_matrix, axis =0)
neg = np.squeeze(np.asarray(neg_tf))
pos = np.squeeze(np.asarray(pos_tf))
neu = np.squeeze(np.asarray(neu_tf))
term_freq_df = pd.DataFrame([neg,pos,neu],columns=cvec.get_feature_names()).transpose()

In [585]:
term_freq_df.columns = ['negative', 'positive','neutral']
term_freq_df['total'] = term_freq_df['negative'] + term_freq_df['positive'] + term_freq_df['neutral']

In [586]:
term_freq_df['date'] = df['date'].min()

In [587]:
term_freq_df.to_csv('Carnival_tokens/Carnival_tokens_w28.csv')

#### Merge files

In [588]:
import glob
import os
path = r'Carnival_tokens'                     
all_files = glob.glob(os.path.join(path, "*.csv"))

In [589]:
all_files

['Carnival_tokens/Carnival_tokens_w1.csv',
 'Carnival_tokens/Carnival_tokens_w27.csv',
 'Carnival_tokens/Carnival_tokens_w26.csv',
 'Carnival_tokens/Carnival_tokens_w2.csv',
 'Carnival_tokens/Carnival_tokens_w24.csv',
 'Carnival_tokens/Carnival_tokens_w18.csv',
 'Carnival_tokens/Carnival_tokens_w19.csv',
 'Carnival_tokens/Carnival_tokens_w25.csv',
 'Carnival_tokens/Carnival_tokens_w3.csv',
 'Carnival_tokens/Carnival_tokens_w7.csv',
 'Carnival_tokens/Carnival_tokens_w21.csv',
 'Carnival_tokens/Carnival_tokens_w20.csv',
 'Carnival_tokens/Carnival_tokens_w6.csv',
 'Carnival_tokens/Carnival_tokens_w4.csv',
 'Carnival_tokens/Carnival_tokens_w22.csv',
 'Carnival_tokens/Carnival_tokens_w23.csv',
 'Carnival_tokens/Carnival_tokens_w5.csv',
 'Carnival_tokens/Carnival_tokens_w8.csv',
 'Carnival_tokens/Carnival_tokens_w12.csv',
 'Carnival_tokens/Carnival_tokens_w13.csv',
 'Carnival_tokens/Carnival_tokens_w9.csv',
 'Carnival_tokens/Carnival_tokens_w11.csv',
 'Carnival_tokens/Carnival_tokens_w10.csv

In [590]:
df_from_each_file = (pd.read_csv(f) for f in all_files)
merge_df   = pd.concat(df_from_each_file, ignore_index=True)

In [591]:
merge_df.to_csv('Carnival_tokens.csv')

In [107]:
df = pd.read_csv('all_general_cleaned_tweets_july.csv')
df1 = pd.read_csv('all_cruiseline_cleaned_tweets_july.csv')

In [111]:
print(df.info())
print(df1.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 283686 entries, 0 to 283685
Data columns (total 12 columns):
date              283686 non-null object
username          283686 non-null object
tweet             283686 non-null object
replies_count     283686 non-null int64
retweets_count    283686 non-null int64
likes_count       283686 non-null float64
hashtags          283686 non-null object
link              283686 non-null object
Cruiseline        283686 non-null object
tidy_tweet        265170 non-null object
Polarity          283686 non-null float64
Analysis          283686 non-null object
dtypes: float64(2), int64(2), object(8)
memory usage: 26.0+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10029 entries, 0 to 10028
Data columns (total 12 columns):
date              10029 non-null object
username          10029 non-null object
tweet             10029 non-null object
replies_count     10029 non-null int64
retweets_count    10029 non-null int64
likes_count       1002

In [109]:
df1 = df1.loc[:, ~df1.columns.str.contains('^Unnamed')]
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

In [110]:
df = df.rename(columns={'company': 'Cruiseline'})

In [112]:
df_final = pd.concat([df,df1])

In [113]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 293715 entries, 0 to 10028
Data columns (total 12 columns):
date              293715 non-null object
username          293715 non-null object
tweet             293715 non-null object
replies_count     293715 non-null int64
retweets_count    293715 non-null int64
likes_count       293715 non-null float64
hashtags          293715 non-null object
link              293715 non-null object
Cruiseline        293715 non-null object
tidy_tweet        274971 non-null object
Polarity          293715 non-null float64
Analysis          293715 non-null object
dtypes: float64(2), int64(2), object(8)
memory usage: 29.1+ MB


In [82]:
#df_final['Cruiseline'] = df_final['company']
#df_final = df_final.drop(['company'],axis = 1)

In [114]:
task_category=['washing','distancing','masks','refunds','cancellation','clean','sanitize','distance','temperature']

In [115]:
df_final['category'] = df_final.tidy_tweet.str.findall('|'.join(task_category)).str[0]

In [116]:
df_final['category'].value_counts()

clean           552
distance        429
cancellation    239
temperature     67 
sanitize        6  
washing         3  
Name: category, dtype: int64

In [187]:
#df_final.to_csv('all_cleaned_may_tweets.csv')

In [117]:
df_final['Cruiseline'].value_counts()

All                283686
Royal Caribbean    3037  
Carnival           1815  
Princess           1014  
Norwegian          795   
Disney             781   
HAL                665   
Viking             545   
Celebrity          466   
Crystal            342   
Azamara            235   
Seabourn           139   
Regent             99    
Oceania            95    
Virgin             1     
Name: Cruiseline, dtype: int64

In [118]:
df_final.head()

,date,username,tweet,replies_count,retweets_count,likes_count,hashtags,link,Cruiseline,tidy_tweet,Polarity,Analysis,category
0,2020-07-01,blatorreinforma,"hear from our global brand & event director, chiara giorgi, while she speaks with insider travel report on bringing the global cruise community together and our new virtual event taking place on october 5-8! #stcglobal #seatrade #virtualevents https://lnkd.in/edzm26x",0,2,2.0,"['#stcglobal', '#seatrade', '#virtualevents']",https://twitter.com/BLaTorreInforma/status/1278425244437549056,All,hear global brand event director insider travel report global community new virtual event take place,0.045455,Positive,NaN
1,2020-07-30,dianainoz2,"my son did philosophy at hs (though he did ib, not hsc) and credits it with why he is cruising through uni. it taught him how to form an argument and look at both sides without ‘cookie cutter approach’ i vote for philosophy over scripture in schools to fix what currently ails us",1,0,5.0,[],https://twitter.com/dianainoz2/status/1288987596501073920,All,son philosophy teach form argument look side cutter approach vote philosophy scripture fix currently,0.000000,Neutral,NaN
2,2020-07-30,josyswagg,"nah not in that house that girl is smart and she is just cruising with everyone , remember she is an artrest",0,0,1.0,[],https://twitter.com/Josyswagg/status/1288987527546839046,All,house girl smart remember,0.214286,Positive,NaN
3,2020-07-30,realjefftidwell,"oh, i think he’s cruising but things will improve. he hasn’t pissed me off lately. i reserve that for gop & dnc!",1,1,2.0,[],https://twitter.com/RealJeffTidwell/status/1288987304456065025,All,think improve lately reserve,-0.300000,Negative,NaN
4,2020-07-30,col_tanya,summer nights were all about cruising that year. pic.twitter.com/rikh15dfho,0,0,0.0,[],https://twitter.com/col_tanya/status/1288987187602694144,All,summer night year pictwittercomrikh15dfho,0.000000,Neutral,NaN


#### Vader analysis

In [119]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

In [120]:
df_final['tidy_tweet'] = df_final['tidy_tweet'].astype('str')

In [121]:
df_final['scores'] = df_final.tidy_tweet.apply(lambda x: analyser.polarity_scores(x))

In [122]:
df_final[['negative', 'neutral', 'positive', 'compund']] = df_final.scores.apply(pd.Series)

In [123]:
def getAnalysis(score):
    if score <= -0.05:
        return 'Negative'
    elif score > -0.05 and score < 0.05:
        return 'Neutral'
    else:
        return 'Positive'

In [124]:
df_final['Analysis_Vader'] = df_final['compund'].apply(getAnalysis)

In [125]:
df_final.head()

,date,username,tweet,replies_count,retweets_count,likes_count,hashtags,link,Cruiseline,tidy_tweet,Polarity,Analysis,category,scores,negative,neutral,positive,compund,Analysis_Vader
0,2020-07-01,blatorreinforma,"hear from our global brand & event director, chiara giorgi, while she speaks with insider travel report on bringing the global cruise community together and our new virtual event taking place on october 5-8! #stcglobal #seatrade #virtualevents https://lnkd.in/edzm26x",0,2,2.0,"['#stcglobal', '#seatrade', '#virtualevents']",https://twitter.com/BLaTorreInforma/status/1278425244437549056,All,hear global brand event director insider travel report global community new virtual event take place,0.045455,Positive,NaN,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}",0.000,1.000,0.000,0.0000,Neutral
1,2020-07-30,dianainoz2,"my son did philosophy at hs (though he did ib, not hsc) and credits it with why he is cruising through uni. it taught him how to form an argument and look at both sides without ‘cookie cutter approach’ i vote for philosophy over scripture in schools to fix what currently ails us",1,0,5.0,[],https://twitter.com/dianainoz2/status/1288987596501073920,All,son philosophy teach form argument look side cutter approach vote philosophy scripture fix currently,0.000000,Neutral,NaN,"{'neg': 0.161, 'neu': 0.839, 'pos': 0.0, 'compound': -0.3612}",0.161,0.839,0.000,-0.3612,Negative
2,2020-07-30,josyswagg,"nah not in that house that girl is smart and she is just cruising with everyone , remember she is an artrest",0,0,1.0,[],https://twitter.com/Josyswagg/status/1288987527546839046,All,house girl smart remember,0.214286,Positive,NaN,"{'neg': 0.0, 'neu': 0.526, 'pos': 0.474, 'compound': 0.4019}",0.000,0.526,0.474,0.4019,Positive
3,2020-07-30,realjefftidwell,"oh, i think he’s cruising but things will improve. he hasn’t pissed me off lately. i reserve that for gop & dnc!",1,1,2.0,[],https://twitter.com/RealJeffTidwell/status/1288987304456065025,All,think improve lately reserve,-0.300000,Negative,NaN,"{'neg': 0.0, 'neu': 0.508, 'pos': 0.492, 'compound': 0.4404}",0.000,0.508,0.492,0.4404,Positive
4,2020-07-30,col_tanya,summer nights were all about cruising that year. pic.twitter.com/rikh15dfho,0,0,0.0,[],https://twitter.com/col_tanya/status/1288987187602694144,All,summer night year pictwittercomrikh15dfho,0.000000,Neutral,NaN,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}",0.000,1.000,0.000,0.0000,Neutral


In [126]:
df_final.to_csv('all_tweets_updated_july.csv')

#### Write results 

In [127]:
file_path = "" # Path to the original transaction file
file_path_parts = "vader_all_analysis/"# Path to transaction file parts

file_name = "all_tweets_updated_july.csv" # Name to the original transaction file
file_name_parts = "cleaned_data_part_july"# Partial_name to transaction file parts

records_per_file = 80000# Records included in a single file

In [128]:
print("Reading file " + file_path + file_name + " ... ")
df_original  = pd.read_csv(file_path + file_name)
df_original = df_original.loc[:, ~df_original.columns.str.contains('^Unnamed')]
print (df_original.shape)
df_original.head()

Reading file all_tweets_updated_july.csv ... 
(293715, 19)


,date,username,tweet,replies_count,retweets_count,likes_count,hashtags,link,Cruiseline,tidy_tweet,Polarity,Analysis,category,scores,negative,neutral,positive,compund,Analysis_Vader
0,2020-07-01,blatorreinforma,"hear from our global brand & event director, chiara giorgi, while she speaks with insider travel report on bringing the global cruise community together and our new virtual event taking place on october 5-8! #stcglobal #seatrade #virtualevents https://lnkd.in/edzm26x",0,2,2.0,"['#stcglobal', '#seatrade', '#virtualevents']",https://twitter.com/BLaTorreInforma/status/1278425244437549056,All,hear global brand event director insider travel report global community new virtual event take place,0.045455,Positive,NaN,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}",0.000,1.000,0.000,0.0000,Neutral
1,2020-07-30,dianainoz2,"my son did philosophy at hs (though he did ib, not hsc) and credits it with why he is cruising through uni. it taught him how to form an argument and look at both sides without ‘cookie cutter approach’ i vote for philosophy over scripture in schools to fix what currently ails us",1,0,5.0,[],https://twitter.com/dianainoz2/status/1288987596501073920,All,son philosophy teach form argument look side cutter approach vote philosophy scripture fix currently,0.000000,Neutral,NaN,"{'neg': 0.161, 'neu': 0.839, 'pos': 0.0, 'compound': -0.3612}",0.161,0.839,0.000,-0.3612,Negative
2,2020-07-30,josyswagg,"nah not in that house that girl is smart and she is just cruising with everyone , remember she is an artrest",0,0,1.0,[],https://twitter.com/Josyswagg/status/1288987527546839046,All,house girl smart remember,0.214286,Positive,NaN,"{'neg': 0.0, 'neu': 0.526, 'pos': 0.474, 'compound': 0.4019}",0.000,0.526,0.474,0.4019,Positive
3,2020-07-30,realjefftidwell,"oh, i think he’s cruising but things will improve. he hasn’t pissed me off lately. i reserve that for gop & dnc!",1,1,2.0,[],https://twitter.com/RealJeffTidwell/status/1288987304456065025,All,think improve lately reserve,-0.300000,Negative,NaN,"{'neg': 0.0, 'neu': 0.508, 'pos': 0.492, 'compound': 0.4404}",0.000,0.508,0.492,0.4404,Positive
4,2020-07-30,col_tanya,summer nights were all about cruising that year. pic.twitter.com/rikh15dfho,0,0,0.0,[],https://twitter.com/col_tanya/status/1288987187602694144,All,summer night year pictwittercomrikh15dfho,0.000000,Neutral,NaN,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}",0.000,1.000,0.000,0.0000,Neutral


In [129]:
i=0
j= records_per_file
size = len(df_original.index)
counter = 1
while i < size:
    if(i + j >= size):
        j = size
    print( "Records " + str(i) + " - " + str(i+j) + " written to file: " + file_name_parts + str(counter).zfill(3) + ".csv")
    df_original[i:(i+j)].to_csv(file_path_parts + file_name_parts + str(counter).zfill(3) + ".csv", sep=',')
    counter+=1
    i = i+j
print ("DONE. Written all records into " + str(counter-1) + " files")

Records 0 - 80000 written to file: cleaned_data_part_july001.csv
Records 80000 - 160000 written to file: cleaned_data_part_july002.csv
Records 160000 - 240000 written to file: cleaned_data_part_july003.csv
Records 240000 - 533715 written to file: cleaned_data_part_july004.csv
DONE. Written all records into 4 files
